In [2]:
__author__ = 'shangxing'
#!/usr/bin/env python
#
#  Python program that will create the sample solution
#
#  @author: lak@climate.com
#  @version: $Id:$
#
#  you may have to install the master branch of 'dask' using:
#     pip install --upgrade git+https://github.com/blaze/dask
#
# Rate from KDP

# From Sachidananda and Zrnic, 1987 (eqn 9).

#   KDP in deg/km.

#     RATE_KDP = sign(KDP) * kdp_aa * (|KDP| ** kdp_bb).

#   where

#     kdp_aa = 40.6
#     kdp_bb = 0.866

import time
import pandas as pd
import numpy as np
import sys
import pickle as df

In [3]:

# change the location of the downloaded test file as necessary.
infile="../data/test.csv"
#infile="kaggle/sample.csv"
outfile="../data/new_solution_11_12.csv"

In [4]:
# read file
t = time.time()
alldata = pd.read_csv(infile)
print 'Time elapsed: {:.4f} secs'.format(time.time()-t) 

Time elapsed: 79.9494 secs


In [12]:
kdp_aa = 40.6
kdp_bb = 0.866

def rainRate_KDP(kdp, minutes_past):
    #print "Estimating rainfall from {0} observations".format(len(minutes_past))
    # how long is each observation valid?
    valid_time = np.zeros_like(minutes_past)
    valid_time[0] = minutes_past.iloc[0]
    for n in range(1, len(minutes_past)):
        valid_time[n] = minutes_past.iloc[n] - minutes_past.iloc[n-1]
    valid_time[-1] = valid_time[-1] + 60 - np.sum(valid_time)
    valid_time = valid_time / 60.0

    # sum up rainrate * validtime
    sum = 0
    for kdp_temp, hours in zip(kdp, valid_time):
        # See: https://en.wikipedia.org/wiki/DBZ_(meteorology)
        if np.isfinite(dbz):
            mmperhr = np.sign(kdp_temp)*kdp_aa*pow(np.abs(kdp_temp),kdp_bb)
            sum = sum + mmperhr * hours
    return sum


# each unique Id is an hour of data at some gauge
def myfunc(hour):

    hour = hour.sort('minutes_past', ascending=True)
    est = rainRate_KDP(hour['Kdp'], hour['minutes_past'])
    return est


#return(sum((sign(Kdp)*(40.6)*(abs(Kdp) **.866)*valid_time), na.rm=TRUE))
print 'Time elapsed: {:.4f} secs'.format(time.time()-t) 

Time elapsed: 46.8730 secs


In [13]:
t = time.time()
estimates = alldata.groupby(alldata.Id).apply(lambda d: myfunc(d))

df = pd.DataFrame(estimates,columns=['Expected'])

df.to_csv(outfile, header=True)
print 'Time elapsed: {:.4f} secs'.format(time.time()-t) 

Time elapsed: 868.7828 secs


/home/jingchu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:24: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [9]:
np.abs(0.5)

0.5